## Gaussian Mixture Model

A Gaussian Micture Model for the credit card fraud problem

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib.gridspec as gridspec
import seaborn as sns

from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,cross_val_predict,cross_val_score, GridSearchCV,RandomizedSearchCV
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import confusion_matrix,classification_report,f1_score,recall_score,precision_score,accuracy_score,precision_recall_curve,roc_curve,roc_auc_score

from collections import Counter

from scipy.stats import norm, multivariate_normal

plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

import warnings
warnings.filterwarnings('ignore')

import random
random.seed(0)

In [ ]:
def Print_Accuracy_Scores(y,y_pred):
    print("F1 Score: ", f1_score(y,y_pred))
    print("Precision Score: ", precision_score(y,y_pred))
    print("Recall Score: ", recall_score(y,y_pred))

In [ ]:
#Loading Dataset
# UNCOMMENT ONE

url = "https://github.com/AET-CS/aet-cs.github.io/blob/main/white/ML/data/creditcard.csv"

# cc_dataset = pd.read_csv(url=url)
# cc_dataset = pd.read_csv("../data/creditcard.csv")

In [ ]:
cc_dataset.drop(labels = ['V28','V27','V26','V25','V24','V23','V22','V20','V15','V13','V8','Time'], axis = 1, inplace=True)
cc_dataset.columns

In [ ]:
cc_dataset.drop(labels = ['V1','V2','V5','V6','V7','V21','Amount'], axis = 1, inplace=True)
cc_dataset.columns

In [ ]:
genuine_data = cc_dataset[cc_dataset['Class']==0]
fraud_data = cc_dataset[cc_dataset['Class']==1]

# optionally reduce data for speed
genuine_data = genuine_data.sample(frac=1, random_state=42)
fraud_data = fraud_data.sample(frac=1.0)

In [ ]:
#Split Genuine records into train & test - 60:40 ratio
genuine_train,genuine_test = train_test_split(genuine_data,test_size=0.4,random_state=0)
print(genuine_train.shape)
print(genuine_test.shape)

In [ ]:
#Split 40% of Genuine Test records into Cross Validation & Test again (50:50 ratio)
genuine_cv,genuine_test = train_test_split(genuine_test,test_size=0.5,random_state=0)
print(genuine_cv.shape)
print(genuine_test.shape)

In [ ]:
#Split Fraud records into Cross Validation & Test (50:50 ratio)
fraud_cv,fraud_test = train_test_split(fraud_data,test_size=0.5,random_state=0)
print(fraud_cv.shape)
print(fraud_test.shape)

In [ ]:
#Drop Y-label from Train data
train_data = genuine_train.drop(labels='Class',axis=1)
print(train_data.shape)

In [ ]:
#Cross validation data
cv_data = pd.concat([genuine_cv,fraud_cv])
cv_data_y = cv_data['Class']
cv_data.drop(labels='Class',axis=1,inplace=True)
print(cv_data.shape)

In [ ]:
#Test data
test_data = pd.concat([genuine_test,fraud_test])
test_data_y = test_data['Class']
test_data.drop(labels='Class',axis=1,inplace=True)
print(test_data.shape)

## The GMM Model

In [ ]:
from sklearn.mixture import GaussianMixture
import numpy as np

# Fit GMM to data (assuming 'data' contains normal instances)
gmm = GaussianMixture(n_components=1, covariance_type='full', random_state=0)
gmm.fit(train_data)

In [ ]:
# Compute the log probability np.of each point
log_probs = gmm.score_samples(train_data)
sns.distplot(log_probs,color='r',label='Valid Class');
plt.title("Log probabilities of training set")

In [ ]:
np.max(log_probs), np.min(log_probs)

In [ ]:
from sklearn.metrics import precision_recall_curve, auc
import matplotlib.pyplot as plt

# Step 1: Compute log-likelihoods for validation data
log_probs_cv = gmm.score_samples(cv_data)  # log probabilities

# Step 2: Generate Precision-Recall data
# Use raw log probabilities as the "scores" for the classifier
precision, recall, thresholds = precision_recall_curve(cv_data_y, -log_probs_cv)  
# Negative log_probs because lower values indicate anomalies

# Step 3: Compute the area under the PR curve
pr_auc = auc(recall, precision)

# Step 4: Plot the Precision-Recall curve
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, marker='.', label=f'GMM (AUC = {pr_auc:.4f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve for GMM Anomaly Detection')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve

# Step 1: Compute log-likelihoods for validation data
log_probs_cv = gmm.score_samples(cv_data)  # log probabilities

# Step 2: Compute precision, recall, and thresholds
precision, recall, thresholds = precision_recall_curve(cv_data_y, -log_probs_cv)

# Step 3: Plot precision and recall vs. threshold
plt.figure(figsize=(8, 6))
plt.plot(thresholds, precision[:-1], label='Precision', color='blue')
plt.plot(thresholds, recall[:-1], label='Recall', color='orange')
plt.xlabel('Threshold (-t)')
plt.ylabel('Score')
plt.title('Precision and Recall vs. Threshold')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
from sklearn.metrics import accuracy_score

t =-30
print("t = ", t)

# Step 1: Compute log-likelihoods for validation data
log_probs_cv = gmm.score_samples(cv_data)  # log probabilities
log_probs_test = gmm.score_samples(test_data)

# Step 2: Classify based on threshold 't'
# Anomaly if log_prob < t → 1 (anomaly), else 0 (normal)
pred_cv = (log_probs_cv < t).astype(int)
pred_test = (log_probs_test < t).astype(int)

# Step 3: Calculate accuracy
accuracy = accuracy_score(cv_data_y, pred_cv)
print(f"Model Accuracy: {accuracy:.4f}")

In [ ]:
cnf_matrix = confusion_matrix(test_data_y, pred_test)
row_sum = cnf_matrix.sum(axis=1,keepdims=True)
cnf_matrix_norm =cnf_matrix  / row_sum 
sns.heatmap(cnf_matrix_norm,cmap='YlGnBu',annot=True)
plt.title("Normalized Confusion Matrix - Test data")
# Set axis labels
plt.xlabel('Predicted')
plt.ylabel('True')

# Display the plot
plt.title('Confusion Matrix Heatmap')
plt.show()

In [ ]:
import numpy as np

# Assuming test_data_y and pred_test are numpy arrays or pandas Series
# Example: test_data_y = np.array([...]), pred_test = np.array([...])

# Compute counts for each category
TP = np.sum((test_data_y == 1) & (pred_test == 1))  # True positives
TN = np.sum((test_data_y == 0) & (pred_test == 0))  # True negatives
FP = np.sum((test_data_y == 0) & (pred_test == 1))  # False positives
FN = np.sum((test_data_y == 1) & (pred_test == 0))  # False negatives

P = np.sum((test_data_y==1))
N = np.sum((test_data_y==0))
LP = np.sum((pred_test == 1))
LN = np.sum((pred_test == 0))
# Total number of samples
total = len(test_data_y)

# Compute percentages
TP_percent = TP / total * 100
TN_percent = TN / total * 100
FP_percent = FP / total * 100
FN_percent = FN / total * 100

# Print the results
print(f"True Positives: {TP} / {P}")
print(f"False Negatives: {FN} / {P}")
print(f"True Negatives: {TN} / {N}")
print(f"False Positives: {FP} / {N}")
